In [1]:
using Compat, Random, Distributions, Plots, LinearAlgebra, Statistics, LaTeXStrings, SparseArrays, CPUTime, JLD2
using SpecialFunctions, GSL, SuiteSparse, Distributed, Roots, NBInclude
gr(fmt=:png)
e = 2.7182818284
@nbinclude("setup.ipynb")
Random.seed!(1);

In [2]:
function qIRF(res_float,res_peg,par)
    plot_ψ = plot(tq_grid,Q*ψ*12*100,color=:black,title=L"\psi_t",xlim=[1,Tq_bar])
    plot_r = plot(tq_grid,Q*(res_float.rd_sequence-ones(L)*res_float.rd_sequence[end])*12*100,color=:deepskyblue,title=L"r_t",xlim=[1,Tq_bar])
    plot!(tq_grid,Q*(res_peg.rd_sequence-ones(L)*res_peg.rd_sequence[end])*12*100,color=:red,ls=:dash,xlim=[1,Tq_bar])
    plot_μ = plot(tq_grid,Q*res_float.μ_sequence*12*100,color=:deepskyblue,title=L"\mu_t",xlim=[1,Tq_bar])
    plot!(tq_grid,Q*res_peg.μ_sequence*12*100,color=:red,ls=:dash,xlim=[1,Tq_bar])
    plot_π = plot(tq_grid,Q*12*res_float.π_sequence*100,color=:deepskyblue,title=L"\pi_t",xlim=[1,Tq_bar])
    plot!(tq_grid,Q*12*res_peg.π_sequence*100,color=:red,ls=:dash,xlim=[1,Tq_bar])

    plot_wN = plot(tq_grid,100*(Q*res_float.wN_sequence-ones(Lq)),color=:deepskyblue,title=L"w^N_t",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.wN_sequence-ones(Lq)),ls=:dash,color=:red,xlim=[1,Tq_bar])
    plot_wT = plot(tq_grid,100*(Q*res_float.wT_sequence-ones(Lq)),color=:deepskyblue,title=L"w^T_t",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.wT_sequence-ones(Lq)),ls=:dash,color=:red,xlim=[1,Tq_bar])
    plot_cN = plot(tq_grid,100*(Q*res_float.cN_sequence/res_float.cN_sequence[end]-ones(Lq)),color=:deepskyblue,title=L"c^N_t",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.cN_sequence/res_peg.cN_sequence[end]-ones(Lq)),ls=:dash,color=:red,xlim=[1,Tq_bar])
    plot_cT = plot(tq_grid,100*(Q*res_float.cT_sequence/res_float.cT_sequence[end]-ones(Lq)),color=:deepskyblue,title=L"c^T_t",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.cT_sequence/res_peg.cT_sequence[end]-ones(Lq)),ls=:dash,color=:red,xlim=[1,Tq_bar])

    plot_lN = plot(tq_grid,100*(Q*res_float.laborN_sequence/res_float.laborN_sequence[end]-ones(Lq)),color=:deepskyblue,title=L"l^N_t",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.laborN_sequence/res_peg.laborN_sequence[end]-ones(Lq)),ls=:dash,color=:red,xlim=[1,Tq_bar])
    plot_lT = plot(tq_grid,100*(Q*res_float.laborT_sequence/res_float.laborT_sequence[end]-ones(Lq)),color=:deepskyblue,title=L"l^T_t",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.laborT_sequence/res_peg.laborT_sequence[end]-ones(Lq)),ls=:dash,color=:red,xlim=[1,Tq_bar])
    plot_aT = plot(tq_grid,100*(Q*res_float.assetT_sequence[1:L]/res_float.assetT_sequence[end]-ones(Lq)),color=:deepskyblue,title=L"a^T_t",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.assetT_sequence[1:L]/res_peg.assetT_sequence[end]-ones(Lq)),ls=:dash,color=:red,xlim=[1,Tq_bar])
    plot_aN = plot(tq_grid,100*(Q*res_float.assetN_sequence[1:L]/res_float.assetN_sequence[end]-ones(Lq)),color=:deepskyblue,title=L"a^N_t",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.assetN_sequence[1:L]/res_peg.assetN_sequence[end]-ones(Lq)),ls=:dash,color=:red,xlim=[1,Tq_bar])
    
    gov_float = res_float.qmain_sequence - (par.ζ_T * res_float.cT_sequence + par.ζ_N * res_float.cN_sequence)
    gov_peg = res_peg.qmain_sequence - (par.ζ_T * res_peg.cT_sequence + par.ζ_N * res_peg.cN_sequence)

    plot_pT = plot(tq_grid,100*(Q*res_float.pT_sequence-ones(Lq)),color=:deepskyblue,title=L"p^T_t",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.pT_sequence-ones(Lq)),ls=:dash,color=:red,xlim=[1,Tq_bar])
    plot_gov = plot(tq_grid,100*(Q*gov_float/gov_float[end]-ones(Lq)),color=:deepskyblue,title=L"g_t",xlim=[1,Tq_bar],ylim=[-12,2])
    plot!(tq_grid,100*(Q*gov_peg/gov_peg[end]-ones(Lq)),ls=:dash,color=:red,xlim=[1,Tq_bar])
    plot_pN = plot(tq_grid,100*(Q*res_float.pN_sequence-ones(Lq)),color=:deepskyblue,title=L"p^N_t",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.pN_sequence-ones(Lq)),ls=:dash,color=:red,xlim=[1,Tq_bar])
    plot_pH = plot(tq_grid,100*(Q*res_float.pH_sequence-ones(Lq)),color=:deepskyblue,title=L"p^H_t",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.pH_sequence-ones(Lq)),ls=:dash,color=:red,xlim=[1,Tq_bar])
    plot_pF = plot(tq_grid,100*(Q*res_float.pF_sequence-ones(Lq)),color=:deepskyblue,title=L"p^F_t",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.pF_sequence-ones(Lq)),ls=:dash,color=:red,xlim=[1,Tq_bar])

    plot_qT = plot(tq_grid,100*(Q*res_float.qT_sequence/q_T-ones(Lq)),color=:deepskyblue,title=L"q_T",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.qT_sequence/q_T-ones(Lq)),ls=:dash,color=:red,label=L"q_T",xlim=[1,Tq_bar])
    plot_qE = plot(tq_grid,100*(Q*res_float.qE_sequence/q_E-ones(Lq)),color=:deepskyblue,title=L"q_E",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.qE_sequence/q_E-ones(Lq)),ls=:dash,color=:red,label=L"q_E",xlim=[1,Tq_bar])
    plot_qH = plot(tq_grid,100*(Q*res_float.qH_sequence/q_H-ones(Lq)),color=:deepskyblue,title=L"q_H",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.qH_sequence/q_H-ones(Lq)),ls=:dash,color=:red,label=L"q_H",xlim=[1,Tq_bar])
    plot_qF = plot(tq_grid,100*(Q*res_float.qF_sequence/q_F-ones(Lq)),color=:deepskyblue,title=L"q_F",xlim=[1,Tq_bar])
    plot!(tq_grid,100*(Q*res_peg.qF_sequence/q_F-ones(Lq)),ls=:dash,color=:red,label=L"q_F",xlim=[1,Tq_bar])

    l = @layout[a a a a; a a a a; a a a a; a a a a; a a a a]
    plot_IRF = plot(plot_ψ,plot_r,plot_μ,plot_π,plot_wN,plot_wT,plot_cN,plot_cT,plot_lN,plot_lT,plot_aN,plot_aT,plot_gov,plot_pT,plot_pH,plot_pF,plot_qT,plot_qE,plot_qH,plot_qF,
        layout=l,size=(900,1250),legend=:false,lw=2)
    return plot_IRF
end

qIRF (generic function with 1 method)

In [3]:
# set the time grid
T = 600
dt = T/1200
T_bar = 24
time_grid = collect(0:dt:T)
L = length(time_grid)
ψ = 0.02/12*exp.(-time_grid/12*1.15)

# set up matrices that convert monthly responses to quarterly
Lq = Int64((L-1)/8)
A = Diagonal(ones(Lq))
b = ones(8)'/8
Q = hcat(kron(A,b),zeros(Lq))
Tq_bar = 11
tq_grid = time_grid[9:8:end]/4

# load parameters
@load "steady_state_output.jld2"

# plot IRF
floats = ["float", "float_low_phil", "float_high_phil", "float_low_elast", "float_high_elast", "float_prop", "float_flat", "float_rank", "float_equal", "float_switched"]
pegs = ["peg", "peg_low_phil", "peg_high_phil", "peg_low_elast", "peg_high_elast", "peg_prop", "peg_flat", "peg_rank", "peg_equal", "peg_switched"]

for i in 1:length(floats)
    res_float, res_peg = [], []
    float_file = "transition_results/"*floats[i]*".jld2"
    @load float_file res_this
    res_float = deepcopy(res_this)
    peg_file = "transition_results/"*pegs[i]*".jld2"
    @load peg_file res_this
    res_peg = deepcopy(res_this)
    if pegs[i] == "peg_rank"
        res_peg.assetN_sequence = ones(length(res_peg.assetN_sequence))
        res_peg.assetT_sequence = ones(length(res_peg.assetN_sequence))
        res_float.assetN_sequence = ones(length(res_peg.assetN_sequence))
        res_float.assetT_sequence = ones(length(res_peg.assetN_sequence))
    end
    if pegs[i] == "peg_equal"
        @load "steady_state_output_equal.jld2"
        plot_IRF = qIRF(res_float,res_peg,par)
        name_png = "graphs/irf"*floats[i][6:end]*".png"
        png(name_png)
    elseif pegs[i] == "peg_switched"
        @load "steady_state_output_switched.jld2"
        plot_IRF = qIRF(res_float,res_peg,par)
        name_png = "graphs/irf"*floats[i][6:end]*".png"
        png(name_png)
    else
        plot_IRF = qIRF(res_float,res_peg,par)
        name_png = "graphs/irf"*floats[i][6:end]*".png"
        png(name_png)
    end
end